# SMART on FHIR

General Resources:  
SMART on FHIR tools, sandboxes, training, etc: https://docs.smarthealthit.org/  
FHIR API tester & U.S. core data in FHIR format https://inferno.healthit.gov/inferno  
Health IT Developer Certification Criteria and registry of certified developers: https://chpl.healthit.gov/#/listing/10591
Cures Act info: https://www.healthit.gov/curesrule/
Policies and technological standards: https://www.cms.gov/Regulations-and-Guidance/Guidance/Interoperability/index  
Epic ONC certification details and cost disclosures: https://www.epic.com/mu

API providers for FHIR:
- [MuleSoft](https://www.mulesoft.com/exchange/68ef9520-24e9-4cf2-b2f5-620025690913/catalyst-accelerator-for-healthcare/). Use pre-built APIs for Epic, Cerner, and others. Maybe faster to get set up and running. 
- [Swagger](https://swagger.io/). Open source DIY API development platform.
- [CyFHIR](https://github.com/Optum/CyFHIR). Write FHIR-formattted data directly to graph and read cypher results back into FHIR.

Potential EHRs for demo:
- [OpenMRS](https://wiki.openmrs.org/display/projects/OpenMRS+HL7+FHIR+Solutions)
- Cerner's SMART on FHIR tutorial/sandbox: https://engineering.cerner.com/smart-on-fhir-tutorial/#introduction
- Epic's SMART on FHIR sandbox and documentation: https://fhir.epic.com/  

FHIR endpoints by healthcare organization: https://open.epic.com/MyApps/Endpoints

In [9]:
import requests
import json

In [130]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j','FHIR'))
session=driver.session()

In [159]:
base = 'https://fhirsandbox.healthit.gov/open/r4/fhir/'
resource = 'Patient/?_id=2&_format=json'
response = requests.get(base+resource)
json_data = response.json()
json_data

{'resourceType': 'Bundle',
 'id': 'f2319e6c-d51d-4041-8541-e2ba492d3f1a',
 'meta': {'lastUpdated': '2022-02-01T12:59:08.773+00:00'},
 'type': 'searchset',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://fhirsandbox.healthit.gov/open/r4/fhir/Patient/?_format=json&_id=2'}]}

In [164]:
base = 'https://fhirsandbox.healthit.gov/open/r4/fhir/'
resource = 'MedicationRequest?patient=1&_format=json'
response = requests.get(base+resource)
json_data = response.json()

# resource = json_data["entry"][0]["resource"]

# rxCUI = resource["medicationCodeableConcept"]["coding"][0]["code"]
# dateWritten = resource["dateWritten"]
# prescriber = resource["prescriber"]["reference"]
# dosageInstruction = resource["dosageInstruction"][0]
# # print("RxNorm CUI:",rxCUI)
# # print("Date:", dateWritten)
# # print("Prescirber:", prescriber)
# # print("Dosage:", dosageInstruction['text'])
# json_data = str(json_data).replace("'", "\\'")
json_data

{'resourceType': 'OperationOutcome',
 'issue': [{'severity': 'error',
   'code': 'processing',
   'diagnostics': "Unknown resource type 'MedicationOrder' - Server knows how to handle: [RiskAssessment, Provenance, Organization, CareTeam, MedicationDispense, DiagnosticReport, ImagingStudy, HealthcareService, Specimen, CarePlan, FamilyMemberHistory, Goal, AllergyIntolerance, StructureDefinition, Medication, Procedure, Encounter, OperationDefinition, DocumentReference, Condition, MedicationRequest, Immunization, Device, Patient, PractitionerRole, ServiceRequest, Practitioner, Observation, MedicationAdministration, MedicationStatement, Location]"}]}

Server knows how to handle:  
RiskAssessment, Provenance, Organization, CareTeam, MedicationDispense, DiagnosticReport, ImagingStudy, HealthcareService, Specimen, CarePlan, FamilyMemberHistory, Goal, AllergyIntolerance, StructureDefinition, Medication, Procedure, Encounter, OperationDefinition, DocumentReference, Condition, MedicationRequest, Immunization, Device, Patient, PractitionerRole, ServiceRequest, Practitioner, Observation, MedicationAdministration, MedicationStatement, Location

In [157]:
command = 'CALL cyfhir.bundle.load("'+json_data+'")'
session.run(command)
# print(command)

To-do:
- Add source vocabulary and source identifier to each node so that we can match to RXCUIs for medications and LOINC codes for other things

In [1]:
listEndpoints = ["Access Community Health Network","Adult & Pediatric Ear, Nose & Throat - Kalamazoo","AdvantageCare Physicians","Adventist Health West","Akron Children's Hospital","Alameda Health System","Allegheny Health Network","Allina Health System","AltaMed","Altru Health System","AnMed Health","Ann & Robert H. Lurie Children's Hospital of Chicago","Ardent","Arizona Community Physicians","Arkansas Children's","Arrowhead Regional Medical Center","Asante Health Systems","Ascension - Providence Healthcare Network","Ascension WI","Aspirus","Atlantic Health","Atrium Health","Atrium Health Wake Forest Baptist","Atrius Health","Aurora Health Care - myAurora","Austin Regional Clinic","Ballad Health","Baptist Health – KY & IN","Baptist Health (Arkansas)","Baptist Memorial Health Care","Bassett Healthcare","BayCare Clinic - myBayCare","Bayhealth Medical Center","Baylor College of Medicine","Baylor Scott & White Health","Beaumont Health","Bellin Health","BJC & Washington University","Bon Secours Health System","Boston Medical Center","Boulder Community Health","Bronson Healthcare Group","Brookwood Baptist Health","Brown & Toland Physicians","Bryan Health","Buffalo Medical Group","Cambridge Health Alliance","Cape Cod Healthcare","Cape Fear Valley Health","Care New England","Carilion Clinic","Carle Foundation Hospital & Physician Group","Caromont Health","Catholic Health (Long Island NY)","Catholic Health System (Buffalo)","Cedars-Sinai Health System","CentraCare Health and Affiliates","Centura Health","Charlotte Eye Ear Nose & Throat Associates","Chesapeake Regional Medical Center","Cheyenne Regional Medical Center","CHI Franciscan Health","CHI Health","CHI St. Luke's Health","CHI St. Vincent","Children's Health System of Texas","Children's Healthcare of Atlanta","Children's Hospital and Medical Center, Omaha Nebraska","Children's Hospital Colorado","Children's Hospital of Philadelphia","Children's Wisconsin","CHRISTUS Health","Cigna Medical Group","Cincinnati Children's Hospital Medical Center","City of Hope","Cleveland Clinic","Columbia Physicians","Columbus Regional Health","Community First Medical Center","Community Health Network","Community Healthcare System","Community Medical Centers","Cone Health","Conemaugh Health System","Confluence Health","Connecticut Children's Medical Center","Contra Costa","Cook Children’s Health Care System","Cooper University Health Care","Cottage Health","Covenant Health","Covenant HealthCare","Dartmouth-Hitchcock","DaVita Physician Solutions","Dayton Children's Hospital","Deaconess Health System","Denver Health","Driscoll Children's Hospital","Duke Health","Duly Health and Care","East Boston Neighborhood Health Center","Edward-Elmhurst Healthcare","Eisenhower Medical Center","El Camino Hospital","El Rio Health","Englewood Hospital and Medical Center","Enloe Medical Center","EPIC Management (Beaver Medical Group)","Erlanger Health System","Eskenazi Health","Essentia Health","Evernorth","Fairview Health Services","Family Health Center (Michigan)","FastMed","FirstHealth of the Carolinas","Franciscan Alliance","Franciscan Missionaries of Our Lady Health System","Fresenius Medical Care North America","Froedtert Health","Garnet Health","Geisinger","Genesis Healthcare System","George Washington University Medical Faculty Associates","Golden Valley Health Centers","Grady Health System","Greater Baltimore Medical Center","Group Health Cooperative - South Central Wisconsin","Gundersen Health System","Hackensack Meridian Health","Harris Health System","Hartford HealthCare","Hattiesburg Clinic and Forrest General Hospital","Hawaii Pacific Health","HCA Central and West Texas","HCA Mountain","HCA South Atlantic","HealthEast Care System","HealthPartners","Heart of Texas Community Health Center","Hendricks Regional Health","Hennepin Healthcare","Henry Ford Health System","Hill Physicians","Hoag Memorial Hospital Presbyterian","HonorHealth","Hospital for Special Surgery","Hospital Sisters Health System (HSHS)","Houston Methodist","Hurley Medical Center","Illinois Bone & Joint Institute","Infirmary Health","Inova and Valley Health","Institute for Family Health","INTEGRIS Health","Jefferson Health","John Muir Health","Johns Hopkins Medicine","JPS Health Network","Kaiser Permanente - Maryland/Virginia/Washington D.C.","Kaiser Permanente - California - Northern","Kaiser Permanente - California - Southern","Kaiser Permanente - Colorado","Kaiser Permanente – Georgia","Kaiser Permanente – Oregon – SW Washington","Kaiser Permanente - Washington","Kaiser Permanente Hawaii / Maui Health System","Kalamazoo College Student Health Center","Kalamazoo Foot Surgery","Kelsey-Seybold Clinic","Kennedy Krieger Institute","Kettering Health Network","King's Daughters Medical Center","Lacy C Kessler, MD, PA","Lahey Health System","Lakeland Health","Lancaster General Health","LCMC Health","Lee Health","Legacy Health","Lehigh Valley Health Network","Leon Medical Centers","Lexington Medical Center","Licking Memorial Health Systems","Lifespan","Loma Linda University Health and CareConnect Partners","Loyola Medicine","Luminis Health","Main Line Health","MaineHealth","Marshall Medical Center","Martin Health System","Mary Greeley Medical Center (Iowa)","Mary Washington Healthcare","Mass General Brigham","Mayo Clinic","McFarland Clinic (Iowa)","Medical University of South Carolina","Medisys Health Network","Memorial Healthcare System","Memorial Hospital and Health Care Center","MemorialCare","Mercy Health - OH, KY","Mercy Health (MO)","Mercy Health Services (MD)","Mercy Health System - WI","Mercy Medical Center","Meritus","Methodist Health System","Methodist Hospitals","Metro Health - Michigan","MetroHealth - OH","Michigan Medicine","Middlesex Hospital","MidMichigan Health","Mohawk Valley Health System","Molina Healthcare","Monroe Clinic","Montage Health","Montefiore Medical Center","Monument Health","Mount Auburn Hospital","Mount Sinai Health System","Mount Sinai Medical Center","MultiCare Health System","Nebraska Medicine","Nemours","Nephrology Center - Southwest Michigan","New Hanover Regional Medical Center","New Jersey Urology","New York-Presbyterian","North Memorial Health","North Mississippi Health Services","North Oaks","Northeast Georgia Health System","NorthShore University Health System","Northwest Community Hospital","Northwestern Medicine","Norton Healthcare","Novant Health","NOVO Health","NYC Health + Hospitals","NYU Langone Medical Center","OB/GYN Associates of Waco - Dr. Rister, Dr. Koeritz","OCHIN","Ochsner Health System","OhioHealth","Olmsted Medical Center","One Brooklyn Health System","Optum ProHEALTH","Oregon Health & Science University","Orlando Health","OrthoCarolina","OrthoVirginia","OSF HealthCare","Our Lady of the Angels","Overlake Hospital Medical Center","Owensboro Health","Pacific Dental Services","Palos Health","Parkland","Parkview Health","PeaceHealth","Pediatric Physicians Organization at Children's","Penn Medicine","Phelps Health","Piedmont Healthcare","Pine Rest Christian Mental Health Services","Planned Parenthood","Premier Health","Presbyterian Healthcare Services","Presence Health","Prime Healthcare","Prisma Health","Prohealth Care","ProMedica Health System","Providence Health & Services - Alaska","Providence Health & Services - Oregon/California","Providence Health & Services - Washington/Montana","QuadMed","Rady Children's","Reid Health","Reliant Medical Group","Renown, Barton, CVMC","River Valley","Riverside Health System (Newport News, VA)","Riverside Medical Center","Riverside Medical Clinic","Riverview Health","Rochester Regional Health","Rush University Medical Center","RWJBarnabas Health","Saint Francis Health System","Saint Francis Healthcare System","Saint Luke's Health System","Salem Health","Salinas Valley Memorial Healthcare Systems","Samaritan Health Services","San Francisco Department of Public Health","Sanford Health","Sansum Clinic","Santa Clara Valley Medical Center Hospitals and Clinics","SCL Health","Scotland Health Care System","Scripps Health","Seattle Children's Hospital","Select Medical","Self Regional Healthcare","Sentara Healthcare","Shannon Health","Singing River Health System","Skagit Regional Health","SolutionHealth","South Georgia Medical Center","South Shore Health System","Southcoast Health","Southeast Health","Southeastern Health","Southern Illinois Healthcare","Sparrow Health System","Spartanburg Regional Healthcare System","Spectrum Health","SSM Health","St. Charles Health System","St. Elizabeth Healthcare","St. Joseph Hospital Health Center","St. Luke’s Health System (Idaho & Eastern Oregon)","St. Luke's Hospital (North Carolina)","St. Luke's University Health Network","Stanford Children's Health","Stanford Health Care","Stormont Vail Health","Summit Health","SUNY Upstate Medical University","Sutter Health","Tahoe Forest Health System","Tampa General Hospital","Tanner Health System","TempleHealth","Texas Children's Hospital","Texas Health Resources","The Brooklyn Hospital Center","The Christ Hospital","The Everett Clinic","The Guthrie Clinic","The Ohio State University Wexner Medical Center","The Portland Clinic","The Queen's Health Systems","The University of Texas Health Science Center at Houston","The University of Texas MD Anderson Cancer Center","The Vancouver Clinic","ThedaCare","TidalHealth","TMC HealthCare","Tower Health","TriHealth","Trinity Health","Trinity Health Of New England","Trinity Health Of New England Medical Group Springfield","Truckee Tahoe Medical Group","UC Davis Health","UC Health","UChicago Medicine","UCLA Medical Center","UConn Health","UCSF Benioff Children's Hospital","UCSF Health","UF Health","UHS San Antonio","UI Health","UMass Memorial Health Care","UMC Southern Nevada","UNC Health Care","United Health Services New York (NYUHS)","United Regional Health Care System","UnityPoint Health","University Health Care System","University Hospital (New Jersey)","University of Arkansas for Medical Sciences","University of California Irvine","University of California San Diego","University of Colorado Health","University of Iowa Health Care","University of Kansas Health System","University of Louisville Physicians","University of Maryland Medical System","University of Miami (UHealth)","University of Mississippi Medical Center","University of Pittsburgh Medical Center (UPMC)","University of Rochester Medical Center","University of Texas Medical Branch","University of Texas Southwestern Medical Center","University of Utah Healthcare","University of Vermont Medical Center","UPMC Pinnacle","UT Health San Antonio","UVA Health System","UW Health And Affiliates - Wisconsin","UW Medicine (Washington)","Valley Children's Healthcare","Valley Medical Center","Valleywise Health","Vanderbilt","VCU Health","Vidant Health","Virginia Hospital Center","Virtua Health","WakeMed Health and Hospitals","Washington Hospital Healthcare System","Watson Clinic","Weill Cornell Medicine","WellSpan Health","WellStar","West Tennessee Healthcare","West Virginia University Medicine","Western Michigan University School of Medicine","Yakima Valley Farm Workers Clinic","Yale New Haven Health System","Yuma Regional Medical Center","Access Community Health Network","Adult & Pediatric Ear, Nose & Throat – Kalamazoo","AdvantageCare Physicians","Adventist Health West","Akron Children's Hospital","Alameda Health System","Allegheny Health Network","Allina Health System","AltaMed","Altru Health System","AnMed Health","Ann & Robert H. Lurie Children's Hospital of Chicago","Ardent","Arizona Community Physicians","Arkansas Children's","Arrowhead Regional Medical Center","Asante Health Systems","Aspirus","Atlantic Health","Atrium Health","Atrium Health Wake Forest Baptist","Atrius Health","Austin Regional Clinic","Baptist Health – KY & IN","Baptist Health (Arkansas)","Baptist Memorial Health Care","Bassett Healthcare","Bayhealth Medical Center","Baylor College of Medicine","Baylor Scott & White","Beaumont Health","Bellin Health","BJC & Washington University","Bon Secours Health System","Boston Medical Center","Boulder Community Health","Bronson Healthcare Group","Brown & Toland Physicians","Bryan Health","Buffalo Medical Group","Cape Fear Valley Health","Care New England","Carilion Clinic","Carle Foundation Hospital & Physician Group","Caromont Health","Catholic Health (Long Island NY)","Catholic Health System (Buffalo)","Cedars-Sinai Health System","CentraCare Health and Affiliates","Charlotte Eye Ear Nose & Throat Associates","Chesapeake Regional Medical Center","Cheyenne Regional Medical Center","Children's Health System of Texas","Children's Hospital and Medical Center, Omaha Nebraska","Children's Hospital Colorado","Children's Hospital of Philadelphia","Children's Wisconsin","Childrens's Healthcare of Atlanta","Cigna Medical Group","Cincinnati Children's Hospital Medical Center","City of Hope","Columbia Physicians","Columbus Regional Health","Community Healthcare System","Community Medical Centers","Cone Health","Conemaugh Health System","Confluence Health","Connecticut Children's Medical Center","Contra Costa","Cook Children’s Health Care System","Cooper University Health Care","Cottage Health","Covenant Health","DaVita Physician Solutions","Dayton Children's Hospital","Deaconess Health System - PRD","Denver Health","Driscoll Children's Hospital","Duke Health","Duly Health and Care","East Boston Neighborhood Health Center","Edward-Elmhurst Healthcare","Eisenhower Medical Center","El Camino Hospital","El Rio Health","Englewood Hospital and Medical Center","Erlanger Health System","Eskenazi Health","Evernorth","Fairview Health Services","Family Health Center (Michigan)","FastMed","FirstHealth of the Carolinas","Franciscan Alliance","Franciscan Missionaries of Our Lady Health System","Fresenius Medical Care North America","Froedtert Health","Garnet Health","Geisinger","Genesis Healthcare System","George Washington University Medical Faculty Associates","Golden Valley Health Centers","Grady Health System","Greater Baltimore Medical Center","Group Health Cooperative - South Central Wisconsin","Gundersen Health System","Hackensack Meridian Health","Hartford HealthCare","Hattiesburg Clinic and Forrest General Hospital","Hawaii Pacific Health - PRD","HCA Central and West Texas","HCA Mountain","HCA South Atlantic","HealthPartners","Hendricks Regional Health","Hennepin Healthcare","Henry Ford Health System","Hill Physicians","Hoag Memorial Hospital Presbyterian","HonorHealth","Hospital for Special Surgery","Hospital Sisters Health System (HSHS)","Houston Methodist","Hurley Medical Center","Illinois Bone & Joint Institute","Infirmary Health","Inova and Valley Health","Institute for Family Health","INTEGRIS Health","Jefferson Health","John Muir Health","Johns Hopkins Medicine","Kaiser Permanente - California - Northern","Kaiser Permanente - California - Southern","Kaiser Permanente - Colorado","Kaiser Permanente – Georgia","Kaiser Permanente - Maryland/Virginia/Washington D.C.","Kaiser Permanente – Oregon – SW Washington","Kaiser Permanente - Washington","Kaiser Permanente Hawaii / Maui Health System","Kalamazoo College Student Health Center","Kalamazoo Foot Surgery","Kelsey-Seybold Clinic","Kennedy Krieger Institute","Kettering Health Network","King's Daughters Medical Center","Lacy C Kessler, MD, PA","Lahey Health System","Lancaster General Health","LCMC Health","Lee Health","Lehigh Valley Health Network","Leon Medical Centers","Licking Memorial Health Systems","Lifespan","Loma Linda University Health and CareConnect Partners","Loyola Medicine","Luminis Health","Main Line Health","MaineHealth","Mary Greeley Medical Center (Iowa)","Mary Washington Healthcare","Mayo Clinic","McFarland Clinic (Iowa)","Medical University of South Carolina","MediSys Health Network","Memorial Healthcare System","Memorial Hospital and Health Care Center","MemorialCare","Mercy Health - OH, KY","Mercy Health System - WI","Mercy Medical Center","Meritus","Methodist Hospitals - PRD","Metro Health - Michigan","MetroHealth - OH","Michigan Medicine","MidMichigan Health","Mohawk Valley Health System","Molina Healthcare","Montage Health","Montefiore Medical Center","Monument Health","Mount Auburn Hospital","Mount Sinai Health System","Mount Sinai Medical Center","Nebraska Medicine","Nemours","Nephrology Center - Southwest Michigan","New Hanover Regional Medical Center","New Jersey Urology","New York-Presbyterian","North Memorial Health","North Mississippi Health Services","Northeast Georgia Health System","NorthShore University Health System","Northwest Community Hospital","Norton Healthcare","Novant Health","NOVO Health","NYC Health + Hospitals","NYU Langone Medical Center","OB/GYN Associates of Waco - Dr. Rister, Dr. Koeritz","OCHIN","Ochsner Health System","OhioHealth","Olmsted Medical Center","One Brooklyn Health System","OptumCare East","Oregon Health & Science University","Orlando Health","OrthoCarolina","OrthoVirginia","OSF HealthCare","Overlake Hospital Medical Center","Owensboro Health","Pacific Dental Services","Palos Health","Parkland","Parkview Health","PeaceHealth","Penn Medicine","Phelps Health","Piedmont Healthcare","Pine Rest Christian Mental Health Services","Planned Parenthood","Premier Health","Presbyterian Healthcare Services","Prisma Health","ProMedica Health System","Providence Health & Services - Alaska","Providence Health & Services - Oregon/California","Providence Health & Services - Washington/Montana","QuadMed","Rady Children's","Reid Health","Reliant Medical Group","Renown, Barton, CVMC","Riverside Health System (Newport News, VA)","Riverside Medical Center","Riverside Medical Clinic","Rochester Regional Health","Rush University Medical Center","RWJBarnabas Health","Saint Francis Health System","Saint Francis Healthcare System (Manual)","Salem Health","Salinas Valley Memorial Healthcare Systems","Samaritan Health Services","San Francisco Department of Public Health","Sanford Health","Sansum Clinic","Santa Clara Valley Medical Center Hospitals and Clinics","SCL Health","Scotland Health Care System","Scripps Health","Seattle Children's Hospital","Select Medical","Self Regional Healthcare","Sentara Healthcare","Shannon Health","Singing River Health System - PRD","Skagit Regional Health","SolutionHealth","South Georgia Medical Center","South Shore Health System","Southeast Health","Southeastern Health","Southern Illinois Healthcare","Spartanburg Regional Health Systems","Spectrum Health","SSM Health","St. Elizabeth Healthcare","St. Joseph Hospital Health Center","St. Luke’s Health System (Idaho & Eastern Oregon)","St. Luke's Hospital (North Carolina)","Stanford Children's Health","Stanford Health Care","Stormont Vail Health","Summit Health","Sutter Health","Tampa General Hospital","Tanner Health System","TempleHealth","Texas Children's","Texas Health Resources","The Brooklyn Hospital Center","The Christ Hospital","The Everett Clinic","The Guthrie Clinic","The Ohio State University Wexner Medical Center","The Portland Clinic","The Queen's Health Systems","The University of Texas Health Science Center at Houston","The University of Texas MD Anderson Cancer Center","ThedaCare","TidalHealth","TMC HealthCare","TriHealth","Trinity Health","Trinity Health of New England","Trinity Health of New England Medical Group Springfield","UC Davis","UC Davis - MMC","UC Health","UChicago Medicine","UCLA Medical Center","UConn Health","UCSF Benioff Children's Hospital","UCSF Health","UF Health","UHS San Antonio","UI Health","UMass Memorial Health Care","UMC Southern Nevada","UNC Health Care","United Health Services New York (NYUHS)","United Regional Health Care System","UnityPoint Health","University Health Care System","University Hospital (New Jersey)","University of Arkansas for Medical Sciences","University of California Irvine","University of California San Diego","University of Iowa Health Care","University of Kansas Health System","University of Louisville Physicians","University of Maryland Medical System","University of Mississippi Medical Center","University of Pittsburgh Medical Center (UPMC)","University of Rochester Medical Center - PRD","University of Texas Medical Branch","University of Texas Southwestern Medical Center","University of Utah Healthcare","University of Vermont Medical Center","UPMC Central PA","UVA Health System","UW Medicine (Washington)","Valley Children's Healthcare","Valleywise Health","Vanderbilt","VCU Health","Virginia Hospital Center","Virtua Health","WakeMed Health and Hospitals","Washington Hospital Healthcare System","Watson Clinic","Weill Cornell Medicine","WellSpan Health","WellStar","West Tennessee Healthcare","West Virginia University Medicine","Western Michigan University School of Medicine","Yakima Valley Farm Workers Clinic","Yale New Haven Health System","Yuma Regional Medical Center"]
print(len(listEndpoints))
setEndpoints = set(listEndpoints)
print(len(setEndpoints))

776
437


# Parse FHIR bundle from Synthea data
Download data from https://inferno.healthit.gov/inferno (see link to test data under the Sample tab). 

Federation scheme:
- at each update from the same organization:
  - if the patient ID exists in the list of previously added patients:
    - isolate the data which existed at the time of the last update and remove its effect from the virtualized population
    - using all of the patients data up to the current time, update all the data fields in the virtualized population
  - if the patient ID does not exist in the previously added patients:
    - update all the data fields in the virtualized population
- log the patient ID and timestamp for the latest data retrieved in a data store for each organization


In [1]:
import json
import pandas as pd

In [2]:
# Get a list of patient bundles in the downloaded data
folder_path = '/media/sata_1TB_internal/uscore-testing-data-12-07-2020/raw/fhir/'

import os
patient_file_list = os.listdir(folder_path) # returns list
patient_file_list[:4]

['Benny518_Olson653_055f50fa-c1cb-c4e5-63db-0d27c307cdd1.json',
 'Jared66_Lang846_d217d366-4496-486f-0c99-834a1c7557e3.json',
 'Harris789_Collins926_d7c34f93-a7d9-df8d-59dc-43c0b542f6c4.json',
 'Marcelo725_Greenholt190_2a27b9a6-2260-1d4a-6a71-a3c4661d0238.json']

In [3]:
# Load a single patient's data
with open(folder_path+patient_file_list[0], 'r') as file:
    data = file.read()

In [4]:
print(data[:800])

{
  "resourceType": "Bundle",
  "type": "transaction",
  "entry": [
    {
      "fullUrl": "urn:uuid:055f50fa-c1cb-c4e5-63db-0d27c307cdd1",
      "resource": {
        "resourceType": "Patient",
        "id": "055f50fa-c1cb-c4e5-63db-0d27c307cdd1",
        "meta": {
          "profile": [
            "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
          ]
        },
        "text": {
          "status": "generated",
          "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</a>.Version identifier: v2.6.1-3-g50f4f58f\n .   Person seed: 1309571695633557482  Population seed: 3</div>"
        },
        "extension": [
          {
            "url": "http://hl7.org/fhir/us/core/StructureDefiniti


In [5]:
json_data = json.loads(data)
len(json_data)

3

In [6]:
json_data.keys()

dict_keys(['resourceType', 'type', 'entry'])

In [7]:
len(json_data['entry'])

1143

In [8]:
# Check the keys of the first few entries in the json_data
for i in range(0, 10):
    print(json_data['entry'][i].keys())

dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])


In [9]:
# Check the keys of the last few entries in the json_data
len_json_data = len(json_data['entry'])

for i in range(len_json_data-10, len_json_data):
    print(json_data['entry'][i].keys())

dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])
dict_keys(['fullUrl', 'resource', 'request'])


In [10]:
json_data['entry'][0]['request']

{'method': 'POST', 'url': 'Patient'}

In [11]:
json_data['entry'][0]['request']['url']

'Patient'

In [12]:
# Get a list of all the request urls
request_url_list = []
for i in range(0, len_json_data):
    request_url_list.append(json_data['entry'][i]['request']['url'])

# Count the unique values from the request url list
pd.Series(request_url_list).value_counts(ascending=True)

Provenance                1
Patient                   1
Location                  3
PractitionerRole          3
Organization              3
Practitioner              3
CarePlan                  6
CareTeam                  6
MedicationRequest         8
Condition                19
Immunization             28
Procedure                37
DocumentReference        71
ExplanationOfBenefit     71
Encounter                71
Claim                    79
DiagnosticReport         97
Observation             636
dtype: int64

The urls in the URL list above are the names of [FHIR resources](http://hl7.org/fhir/resourcelist.html). To conceive of this data in a labeled property graph, each resource type can be a node label, each instance of the resource can be a node, and the relevant details of each resource can be node properties. 

In [19]:
for i in range(0, len_json_data):
    if json_data['entry'][i]['request']['url'] == "Observation":
        sample_obs = json_data['entry'][i]
        break
sample_obs

{'fullUrl': 'urn:uuid:4ea2eeb0-81d3-0659-6672-7381c50b1fc9',
 'resource': {'resourceType': 'Observation',
  'id': '4ea2eeb0-81d3-0659-6672-7381c50b1fc9',
  'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/bodyheight',
    'http://hl7.org/fhir/StructureDefinition/vitalsigns']},
  'status': 'final',
  'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
      'code': 'vital-signs',
      'display': 'vital-signs'}]}],
  'code': {'coding': [{'system': 'http://loinc.org',
     'code': '8302-2',
     'display': 'Body Height'}],
   'text': 'Body Height'},
  'subject': {'reference': 'urn:uuid:055f50fa-c1cb-c4e5-63db-0d27c307cdd1'},
  'encounter': {'reference': 'urn:uuid:a075d2c5-90dc-1f14-0ecc-8e6d589f43a7'},
  'effectiveDateTime': '1940-06-27T15:02:00-04:00',
  'issued': '1940-06-27T15:02:00.214-04:00',
  'valueQuantity': {'value': 55.6,
   'unit': 'cm',
   'system': 'http://unitsofmeasure.org',
   'code': 'cm'}},
 'request': {'method'

In [20]:
sample_obs['resource'].keys()

dict_keys(['resourceType', 'id', 'meta', 'status', 'category', 'code', 'subject', 'encounter', 'effectiveDateTime', 'issued', 'valueQuantity'])

In [28]:
sample_obs['resource']['code']

{'coding': [{'system': 'http://loinc.org',
   'code': '8302-2',
   'display': 'Body Height'}],
 'text': 'Body Height'}

In [30]:
sample_obs['resource']['code']['text']

'Body Height'

In [32]:
observation_types_list = []
for i in range(0, len_json_data):
    if json_data['entry'][i]['request']['url'] == "Observation":
        observation_type = json_data['entry'][i]['resource']['code']['text']
        observation_types_list.append(observation_type)

# Count the unique values from the request url list
pd.Series(observation_types_list).value_counts()

Pain severity - 0-10 verbal numeric rating [Score] - Reported                45
Body Weight                                                                  39
Heart rate                                                                   39
Tobacco smoking status NHIS                                                  39
Body Height                                                                  39
Blood Pressure                                                               39
Respiratory rate                                                             39
Body Mass Index                                                              30
Body mass index (BMI) [Percentile] Per age and gender                        21
MCH [Entitic mass] by Automated count                                        15
Platelet distribution width [Entitic volume] in Blood by Automated count     15
Hemoglobin [Mass/volume] in Blood                                            15
Platelets [#/volume] in Blood by Automat

In [35]:
body_weight_list = []
for i in range(0, len_json_data):
    try:
        if json_data['entry'][i]['resource']['code']['text'] == "Body Weight":
            body_weight = json_data['entry'][i]['resource']['valueQuantity']['value']
            unit = json_data['entry'][i]['resource']['valueQuantity']['unit']
            body_weight_list.append(body_weight)
    except:
        pass
print(body_weight_list)
print(unit)

[4.6, 5.6, 7.3, 8.7, 10.3, 11.5, 12.4, 13.1, 14.2, 16.1, 18.9, 20.3, 20.2, 20.7, 23.1, 26.5, 29.3, 33.3, 35.5, 40, 47.6, 48.8, 53.1, 62.7, 71.3, 74.1, 77.2, 80.6, 91.9, 91.2, 96.2, 101.3, 97.6, 101.2, 98.3, 93.6, 100.2, 95.4, 98.6]
kg


In [40]:
observation_types_list = []
observation_values_list = []
observation_units_list = []
observation_effectiveDateTime_list = []

for i in range(0, len_json_data):
    if json_data['entry'][i]['request']['url'] == "Observation":
        
        try:
            observation_type = json_data['entry'][i]['resource']['code']['text']
            observation_value = json_data['entry'][i]['resource']['valueQuantity']['value']
            observation_unit = json_data['entry'][i]['resource']['valueQuantity']['unit']
            effectiveDateTime = json_data['entry'][i]['resource']['effectiveDateTime']

            observation_types_list.append(observation_type)
            observation_values_list.append(observation_value)
            observation_units_list.append(observation_unit)
            observation_effectiveDateTime_list.append(effectiveDateTime)
            
        except:
            pass

# Count the unique values from the request url list
quantitative_observations = pd.DataFrame([observation_types_list, observation_values_list, observation_units_list, observation_effectiveDateTime_list]).transpose()
quantitative_observations

,0,1,2,3
0,Body Height,55.6,cm,1940-06-27T15:02:00-04:00
1,Pain severity - 0-10 verbal numeric rating [Sc...,2,{score},1940-06-27T15:02:00-04:00
2,Body Weight,4.6,kg,1940-06-27T15:02:00-04:00
3,Weight-for-length Per age and sex,18.498,%,1940-06-27T15:02:00-04:00
4,Head Occipital-frontal circumference,37.73,cm,1940-06-27T15:02:00-04:00
...,...,...,...,...
552,Pain severity - 0-10 verbal numeric rating [Sc...,1,{score},1984-07-12T15:02:00-04:00
553,Body Weight,98.6,kg,1984-07-12T15:02:00-04:00
554,Body Mass Index,28.85,kg/m2,1984-07-12T15:02:00-04:00
555,Heart rate,96,/min,1984-07-12T15:02:00-04:00


In [ ]:
for i in range(0, len_json_data):
    request_url_list.append(json_data['entry'][i]['request']['url'])